由于文件 Tencent_AILab_ChineseEmbedding.txt 太大，需要先到 https://ai.tencent.com/ailab/nlp/zh/embedding.html 下载，并解压出该文件到同一目录下才可运行本代码

In [5]:
# 定义两个list分别存放两个板块的帖子数据
import jieba
academy_titles = []
job_titles = []
with open('academy_titles.txt', encoding='utf8') as f:
    for l in f:  # 按行读取文件
        academy_titles.append(list(jieba.cut(l.strip( ))))  # strip 方法用于去掉行尾空格
with open('job_titles.txt', encoding='utf8') as f:
    for l in f:  # 按行读取文件
        job_titles.append(list(jieba.cut(l.strip( ))))  # strip 方法用于去掉行尾空格

In [6]:
academy_titles[2]

['出售', '人大', '新闻', '学院', '2015', '年', '考研', '权威', '资料']

In [7]:
word_set = set()
for title in academy_titles:
    for word in title:
        word_set.add(word.lower())
for title in job_titles:
    for word in title:
        word_set.add(word.lower())
print(len(word_set))

4059


In [8]:
from tqdm import tqdm
f = open('Tencent_AILab_ChineseEmbedding.txt', encoding='utf8')
word2v = {}
wl = []
for l in tqdm(f):
    l = l.strip().split(' ')
    wl.append(l[0])
    if l[0] in word_set:
        word2v[l[0]] = list(map(float, l[1:]))

FileNotFoundError: [Errno 2] No such file or directory: 'Tencent_AILab_ChineseEmbedding.txt'

In [ ]:
len(word2v)

In [ ]:
import json
with open('word2v', 'r') as f:
    word2v = json.load(f)

In [ ]:
len(word_set)

In [ ]:
4085-3924

In [ ]:
dx = []
for wd in word_set:
    if wd.lower() not in word2v:
        dx.append(wd)

In [ ]:
len(dx)

In [ ]:
len(wl)

In [ ]:
'NLP'.lower() in wl

In [ ]:
dx

In [ ]:
import torch

def title_to_tensor(title):
    words_vectors = []
    for word in title:
        if word in word2v:
            words_vectors.append(word2v[word])
    tensor = torch.tensor(words_vectors, dtype=torch.float)
    return tensor

In [ ]:
import torch.nn as nn

class RNN(nn.Module):
    def __init__(self, embedding_size, hidden_size, output_size):
        super(RNN, self).__init__()
        self.hidden_size = hidden_size
        self.i2h = nn.Linear(embedding_size + hidden_size, hidden_size)
        self.i2o = nn.Linear(embedding_size + hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input_tensor, hidden):
        word_vector = input_tensor
        combined = torch.cat((word_vector, hidden), 1)
        hidden = self.i2h(combined)
        output = self.i2o(combined)
        output = self.softmax(output)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, self.hidden_size)

embedding_size = 200
n_hidden = 128
n_categories = 2
rnn = RNN(embedding_size, n_hidden, n_categories)

In [ ]:
input_tensor = title_to_tensor(academy_titles[0])
print('input_tensor:\n', input_tensor)

In [ ]:
hidden = rnn.initHidden()
output, hidden = rnn(input_tensor[0].unsqueeze(dim=0), hidden)
print('output:\n', output)
print('hidden:\n', hidden)
print('size of hidden:\n', hidden.size())

In [ ]:
def run_rnn(rnn, input_tensor):
    hidden = rnn.initHidden()
    for i in range(input_tensor.size()[0]):
        output, hidden = rnn(input_tensor[i].unsqueeze(dim=0), hidden)
    return output

In [ ]:
all_data = []
categories = ["考研考博", "招聘信息"]

In [ ]:
for l in academy_titles:
    all_data.append((title_to_tensor(l), torch.tensor([0], dtype=torch.long)))
for l in job_titles:
    all_data.append((title_to_tensor(l), torch.tensor([1], dtype=torch.long)))

In [ ]:
import random
random.shuffle(all_data)
data_len = len(all_data)
split_ratio = 0.7
train_data = all_data[:int(data_len*split_ratio)]
test_data = all_data[int(data_len*split_ratio):]
print("Train data size: ", len(train_data))
print("Test data size: ", len(test_data))

In [ ]:
def train(rnn, criterion, input_tensor, category_tensor):
    rnn.zero_grad()
    output = run_rnn(rnn, input_tensor)
    loss = criterion(output, category_tensor)
    loss.backward()

    # 根据梯度更新模型的参数
    for p in rnn.parameters():
        p.data.add_(p.grad.data, alpha=-learning_rate)

    return output, loss.item()
    

In [ ]:
def evaluate(rnn, input_tensor):
    with torch.no_grad():
        hidden = rnn.initHidden()
        output = run_rnn(rnn, input_tensor)
        return output

In [ ]:
from tqdm import tqdm
epoch = 1
embedding_size = 200
n_hidden = 10
n_categories = 2
learning_rate = 0.005
rnn = RNN(embedding_size, n_hidden, n_categories)
criterion = nn.NLLLoss()
loss_sum = 0
all_losses = []
plot_every = 100
for e in range(epoch):
    for ind, (title_tensor, label) in enumerate(tqdm(train_data)):
        if len(title_tensor) == 0:
            continue
        output, loss = train(rnn, criterion, title_tensor, label)
        loss_sum += loss
        if ind % plot_every == 0:
            all_losses.append(loss_sum / plot_every)
            loss_sum = 0
    c = 0
    for title, category in tqdm(test_data):
        if len(title) == 0:
            continue
        output = evaluate(rnn, title)
        topn, topi = output.topk(1)
        if topi.item() == category[0].item():
            c += 1
    print('accuracy', c / len(test_data))

In [ ]:
c = 0
l1 = []
l2 = []
for title, category in tqdm(test_data):
    output = evaluate(rnn, title)
    topn, topi = output.topk(1)
    l1.append(topi.item())
    l2.append(category[0].item())
    if topi.item() == category[0].item():
        c += 1
print('accuracy', c / len(test_data))

In [ ]:
print(l1[:40])
print(l2[:40])

In [ ]:
sum(l1)

In [ ]:
sum(l2)

In [ ]:
c = 0
for title, category in tqdm(test_data):
    output = evaluate(rnn, title)
    topn, topi = output.topk(1)
    if topi.item() == category[0].item():
        c += 1
print('accuracy', c / len(test_data))

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(10,7))
plt.ylabel('Average Loss')
plt.plot(all_losses[1:])

In [ ]:
torch.save(rnn, 'rnn_model.pkl')

In [ ]:
def get_category(rnn, title):
    input_tensor = title_to_tensor(title)
    with torch.no_grad():
        hidden = rnn.initHidden()
        output = run_rnn(rnn, input_tensor)
        topv, topi = output.topk(1)
        return categories[topi.item()]

In [ ]:
input_tensor = title_to_tensor("北大实验室招硕博连读保研学生")

In [ ]:
def get_category(title):
    title = title_to_tensor(title)
    output = evaluate(rnn, title)
    topn, topi = output.topk(1)
    return categories[topi.item()]

In [ ]:
print("【校招】今日头条后端开发工程师", get_category(rnn, "【校招】今日头条后端开发工程师"))

In [ ]:
print("毕业找房子", get_category(rnn, "毕业找房子"))

In [ ]:

print("学校附近的公寓", get_category(rnn, "学校附近的公寓"))
print("学校附近的公寓", get_category(rnn, "学校附近的公寓"))
print("考博经验帖", get_category(rnn, "考博经验帖"))
print("2021年秋季出国交流", get_category(rnn, "2021年秋季出国交流"))
print("考研学校选择，纠结，求师哥师姐指导", get_category(rnn, "考研学校选择，纠结，求师哥师姐指导"))